In [23]:
import pandas as pd

coronavirus_posts = pd.read_csv("../application_dataset/coronavirus1.csv")
coronavirus_posts2 = coronavirus_posts.drop(coronavirus_posts.columns[0], axis=1)
coronavirus_posts_list = coronavirus_posts[coronavirus_posts2.columns[0]].values.tolist()

In [24]:
import re

def remove_numbers(post):
    return re.sub(r'\d+', '', post)

coronavirus_posts_list = [remove_numbers(p) for p in coronavirus_posts_list]

import string

def remove_punctuation(post):
    translator = str.maketrans('', '', string.punctuation)
    return post.translate(translator)

coronavirus_posts_list = [remove_punctuation(p) for p in coronavirus_posts_list]

def remove_whitespace(post):
    return " ".join(post.split())

coronavirus_posts_list = [remove_whitespace(p) for p in coronavirus_posts_list]

In [25]:
# get stop word lists
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to C:\Users\ShiYu
[nltk_data]     Qiu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
from nltk.tokenize import TweetTokenizer

tweetTokenizer = TweetTokenizer()

In [27]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

stemmer = WordNetLemmatizer()

def tokenizer(sentence):
    return (stemmer.lemmatize(w, pos='v') for w in tweetTokenizer.tokenize(sentence.lower()) if w not in stop_words)

[nltk_data] Downloading package wordnet to C:\Users\ShiYu
[nltk_data]     Qiu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
import numpy
coronavirus_posts_list = [tokenizer(s) for s in coronavirus_posts_list]
coronavirus_posts_list = [list(s) for s in coronavirus_posts_list]
#from gensim.models import Word2Vec
#path=get_tmpfile("../preprocess/word2vec.model")
#model = Word2Vec.load('../preprocess/word2vec.model')

NameError: name 'get_tmpfile' is not defined

In [29]:
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
path=get_tmpfile("../preprocess/word2vec.model")
model = Word2Vec.load('../preprocess/word2vec.model')

ModuleNotFoundError: No module named 'numpy.random._pickle'

In [7]:
import numpy as np
# Function to average all word vectors in a post
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# Function for calculating the average feature vector
def getAvgFeatureVecs(tweets, model, num_features):
    counter = 0
    tweetFeatureVecs = np.zeros((len(tweets),num_features),dtype="float32")
    for tweet in tweets:
        # Printing a status message every 1000th tweets
        #if counter%1000 == 0:
        #    print("Review %d of %d"%(counter,len(tweets)))
            
        tweetFeatureVecs[counter] = featureVecMethod(tweet, model, num_features)
        counter = counter+1
        
    return tweetFeatureVecs

In [8]:
trainDataVecs = getAvgFeatureVecs(coronavirus_posts_list, model, 100)
trainDataVecs=np.nan_to_num(trainDataVecs)
df = pd.DataFrame(trainDataVecs)
df.to_csv('../new_data/coronavirus_word2vec.csv',index=False)

C:\Users\ShiYu Qiu\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
